# XML, HTML, and Web Scraping

JSON and XML are two different ways to represent hierarchical data. Which one is better? There are lots of articles online which discuss similarities and differences between JSON and XML and their advantages and disadvantages. Both formats are still in current usage, so it is good to be familiar with both. However, JSON is more common, so we'll focus on working with JSON representations of hierarchical data.

The reading covered an example of using Beautiful Soup to parse XML. Rather than doing another example XML now, we'll skip straight to scraping HTML from a webpage. Both HTML and XML can be parsed in a similar way with Beautiful Soup.

In [1]:
import pandas as pd

## Scraping an HTML table with Beautiful Soup

Open the URL https://en.wikipedia.org/wiki/List_of_United_States_cities_by_population and scroll down until you see a table of the cities in the U.S. with population over 100,000 (as of Jul 1, 2022). We'll use Beautiful Soup to scrape information from this table.

Read in the HTML from the ULR using the `requests` library.

In [5]:
url = "https://en.wikipedia.org/wiki/List_of_United_States_cities_by_population"
headers = {"User-Agent": "Mozilla/5.0"}
response = requests.get(url, headers=headers)
html = response.text
print(html[:500])




<!DOCTYPE html>
<html class="client-nojs vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-limited-width-clientpref-1 vector-feature-limited-width-content-enabled vector-feature-custom-font-size-clientpref-1 vector-feature-appearance-pinned-clientpref-1 vector-feature-night-mode-enabled skin-theme-clientpref-day vect


Use Beautiful Soup to parse this string into a tree called `soup`

In [6]:
soup = BeautifulSoup(html, "lxml")
soup.title.get_text()


'List of United States cities by population - Wikipedia'

To find an HTML tag corresponding to a specific element on a webpage, right-click on it and choose "Inspect element". Go to the cities table Wikipedia page and do this now.

You should find that the cities table on the Wikipedia page corresponds to the element

```
<table class="wikitable sortable jquery-tablesorter" style="text-align:center">
```

There are many `<table>` tags on the page.

In [7]:
len(soup.find_all("table"))

10

We can use attributes like `class=` and `style=` to narrow down the list.

In [8]:
len(soup.find_all("table",
                  attrs={
                      "class": "wikitable sortable",
                      "style": "text-align:center"}
                  ))

0

At this point, you can manually inspect the tables on the webpage to find that the one we want is the first one (see `[0]` below). We'll store this as `table`.

In [ ]:
table = soup.find_all("table",
                  attrs={
                      "class": "wikitable sortable",
                      "style": "text-align:center"}
                  )[0]

**Now you will write code to scrape the information in `table` to create a Pandas data frame with one row for each city and columns for: city, state, population (2022 estimate), and 2020 land area (sq mi).** Refer to the Notes/suggestions below as you write your code. A few Hints are provided further down, but try coding first before looking at the hints.

Notes/suggestions:

- Use as a guide the code from the reading that produced the data frame of Statistics faculty
- Inspect the page source as you write your code
- You will need to write a loop to get the information for all cities, but you might want to try just scraping the info for New York first
- You will need to pull the text from the tag. If `.text` returns text with "\n" at the end, try `.get_text(strip = True)` instead of `.text`
- Don't forget to convert to a Pandas Data Frame; it should have 333 rows and 4 columns
- The goal of this exercise is just to create the Data Frame. If you were going to use it --- e.g., what is the population density for all cities in CA? --- then you would need to clean the data first (to clean strings and convert to quantitative). (You can use Beautiful Soup to do some of the cleaning for you, but that goes beyond our scope.)

In [ ]:
# YOUR CODE HERE. ADD AS MANY CELLS AS NEEDED

In [13]:
tables = soup.find_all("table")
target = None
for t in tables:
    hdrs = " ".join(th.get_text(strip=True).lower() for th in t.find_all("th"))
    if "city" in hdrs and "land area" in hdrs and "estimate" in hdrs:
        target = t
        break

if target is None:
    raise ValueError("No matching table found")

rows = []
for tr in target.find_all("tr")[1:]:
    cells = tr.find_all(["td","th"])
    if len(cells) < 7:
        continue
    city = cells[1].get_text(strip=True)
    state = cells[2].get_text(strip=True)
    pop = cells[3].get_text(strip=True)
    land = cells[6].get_text(strip=True)
    rows.append({
        "city": city,
        "state": state,
        "population_estimate": pop,
        "land_area_sq_mi_2020": land
    })

df_cities = pd.DataFrame(rows, columns=["city","state","population_estimate","land_area_sq_mi_2020"])
df_cities.shape, df_cities.head()


((346, 4),
   city      state population_estimate land_area_sq_mi_2020
 0   NY  8,478,072           8,804,190                778.3
 1   CA  3,878,704           3,898,747              1,216.0
 2   IL  2,721,308           2,746,388                589.7
 3   TX  2,390,125           2,304,580              1,658.6
 4   AZ  1,673,164           1,608,139              1,341.6)

Hints:

- Each city is a row in the table; find all the `<tr>` tags to find all the cities
- Look for the `<td>` tag to see table entries within a row
- The rank column is represented by `<th>` tags, rather than `<td>` tags. So within a row, the first (that is, `[0]`) `<td>` tag corresponds to the city name.

## Aside: Scraping an HTML table with Pandas



The Pandas command `read_html` can be used to scrape information from an HTML table on a webpage.

We can call `read_html` on the URL.

In [18]:
import requests, pandas as pd
from io import StringIO

url = "https://en.wikipedia.org/wiki/List_of_United_States_cities_by_population"
html = requests.get(url, headers={"User-Agent": "Mozilla/5.0"}).text
tables = pd.read_html(StringIO(html), flavor="lxml")

pick = None
for df in tables:
    cols = [str(c).lower() for c in df.columns]
    if any("city" in c for c in cols) and any("land area" in c for c in cols):
        pick = df
        break

df_cities2 = pick
df_cities2.shape, df_cities2.head()



((346, 10),
           City  ST 2024 estimate 2020 census  Change 2020 land area          \
           City  ST 2024 estimate 2020 census  Change            mi2     km2   
 0  New York[c]  NY       8478072     8804190  −3.70%          300.5   778.3   
 1  Los Angeles  CA       3878704     3898747  −0.51%          469.5  1216.0   
 2      Chicago  IL       2721308     2746388  −0.91%          227.7   589.7   
 3      Houston  TX       2390125     2304580  +3.71%          640.4  1658.6   
 4      Phoenix  AZ       1673164     1608139  +4.04%          518.0  1341.6   
 
   2020 density                                      Location  
          / mi2  / km2                               Location  
 0        29298  11312    40°40′N 73°56′W﻿ / ﻿40.66°N 73.94°W  
 1         8304   3206  34°01′N 118°25′W﻿ / ﻿34.02°N 118.41°W  
 2        12061   4657    41°50′N 87°41′W﻿ / ﻿41.84°N 87.68°W  
 3         3599   1390    29°47′N 95°23′W﻿ / ﻿29.79°N 95.39°W  
 4         3105   1199  33°34′N 112°05′W﻿ 

However, this scrapes all the tables on the webpage, not just the one we want. As with Beautiful Soup, we can narrow the search by specifying the table attributes.

In [21]:
import requests, pandas as pd
from io import StringIO

url = "https://en.wikipedia.org/wiki/List_of_United_States_cities_by_population"
html = requests.get(url, headers={"User-Agent": "Mozilla/5.0"}).text

tables = pd.read_html(StringIO(html), flavor="lxml")
df_cities2 = next(
    df for df in tables
    if any("city" in str(c).lower() for c in df.columns)
    and any("land area" in str(c).lower() for c in df.columns)
)
df_cities2.shape, df_cities2.head()



((346, 10),
           City  ST 2024 estimate 2020 census  Change 2020 land area          \
           City  ST 2024 estimate 2020 census  Change            mi2     km2   
 0  New York[c]  NY       8478072     8804190  −3.70%          300.5   778.3   
 1  Los Angeles  CA       3878704     3898747  −0.51%          469.5  1216.0   
 2      Chicago  IL       2721308     2746388  −0.91%          227.7   589.7   
 3      Houston  TX       2390125     2304580  +3.71%          640.4  1658.6   
 4      Phoenix  AZ       1673164     1608139  +4.04%          518.0  1341.6   
 
   2020 density                                      Location  
          / mi2  / km2                               Location  
 0        29298  11312    40°40′N 73°56′W﻿ / ﻿40.66°N 73.94°W  
 1         8304   3206  34°01′N 118°25′W﻿ / ﻿34.02°N 118.41°W  
 2        12061   4657    41°50′N 87°41′W﻿ / ﻿41.84°N 87.68°W  
 3         3599   1390    29°47′N 95°23′W﻿ / ﻿29.79°N 95.39°W  
 4         3105   1199  33°34′N 112°05′W﻿ 

This still returns 3 tables. As we remarked above, the table that we want is the first one (see `[0]` below).

In [24]:
import requests
import pandas as pd
from io import StringIO

url = "https://en.wikipedia.org/wiki/List_of_United_States_cities_by_population"
html = requests.get(url, headers={"User-Agent": "Mozilla/5.0"}).text
tables = pd.read_html(StringIO(html), flavor="lxml")

for i, t in enumerate(tables):
    cols = [str(c).lower() for c in t.columns]
    if any("city" in c for c in cols) and any("land area" in c for c in cols):
        df_cities2 = t
        print("✅ found table index:", i)
        break

df_cities2.shape, df_cities2.head()



✅ found table index: 2


((346, 10),
           City  ST 2024 estimate 2020 census  Change 2020 land area          \
           City  ST 2024 estimate 2020 census  Change            mi2     km2   
 0  New York[c]  NY       8478072     8804190  −3.70%          300.5   778.3   
 1  Los Angeles  CA       3878704     3898747  −0.51%          469.5  1216.0   
 2      Chicago  IL       2721308     2746388  −0.91%          227.7   589.7   
 3      Houston  TX       2390125     2304580  +3.71%          640.4  1658.6   
 4      Phoenix  AZ       1673164     1608139  +4.04%          518.0  1341.6   
 
   2020 density                                      Location  
          / mi2  / km2                               Location  
 0        29298  11312    40°40′N 73°56′W﻿ / ﻿40.66°N 73.94°W  
 1         8304   3206  34°01′N 118°25′W﻿ / ﻿34.02°N 118.41°W  
 2        12061   4657    41°50′N 87°41′W﻿ / ﻿41.84°N 87.68°W  
 3         3599   1390    29°47′N 95°23′W﻿ / ﻿29.79°N 95.39°W  
 4         3105   1199  33°34′N 112°05′W﻿ 

Wait, that seemed much easier than using Beautiful Soup, and it returned a data frame, and we even got for free some formatting like removing the commas from the population! Why didn't we just use `read_html` in the first place? It's true the `read_html` works well when scraping information from an HTML *table*. Unfortunately, you often want to scrape information from a webpage that isn't conveniently stored in an HTML table, in which case `read_html` won't work. (It only searches for `<table>`, `<th>`, `<tr>`, and `<td>` tags, but there are many other HTML tags.) Though Beautiful Soup is not as simple as `read_html`, it is more flexible and thus more widely applicable.

## Scraping information that is NOT in a `<table>` with Beautiful Soup

The Cal Poly course catalog http://catalog.calpoly.edu/collegesandprograms/collegeofsciencemathematics/statistics/#courseinventory contains a list of courses offered by the Statistics department. **You will scrape this website to obtain a Pandas data frame with one row for each DATA or STAT course and two columns: course name and number (e.g, DATA 301. Introduction to Data Science) and term typically offered (e.g., Term Typically Offered: F, W, SP).**

Note: Pandas `read_html` is not help here since the courses are not stored in a `<table>.`

In [25]:
pd.read_html("http://catalog.calpoly.edu/collegesandprograms/collegeofsciencemathematics/statistics/#courseinventory")

[                                        Program name   Program type
 0                              Actuarial Preparation          Minor
 1  Cross Disciplinary Studies Minor in Bioinforma...          Minor
 2   Cross Disciplinary Studies Minor in Data Science          Minor
 3                                         Statistics  BS, MS, Minor]


Notes/suggestions:


- Inspect the page source as you write your code
- The courses are not stored in a `<table>`. How are they stored?
- You will need to write a loop to get the information for all courses, but you might want to try just scraping the info for DATA 100 first
- What kind of tag is the course name stored in? What is the `class` of the tag?
- What kind of tag is the quarter(s) the course is offered stored in? What is the `class` of the tag? Is this the only tag of this type with the class? How will you get the one you want?
- You don't have to remove the number of units (e.g., 4 units) from the course name and number, but you can try it if you want
- You will need to pull the text from the tag. If `.text` returns text with "\n" at the end, try `get_text(strip = True)` instead of `text`
- Don't forget to convert to a Pandas Data Frame; it should have 74 rows and 2 columns
- The goal of this exercise is just to create the Data Frame. If you were going to use it then you might need to clean the data first. (You can use Beautiful Soup to do some of the cleaning for you, but that goes beyond our scope.)



In [ ]:
# YOUR CODE HERE. ADD AS MANY CELLS AS NEEDED

In [26]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

url = "http://catalog.calpoly.edu/collegesandprograms/collegeofsciencemathematics/statistics/#courseinventory"
html = requests.get(url, headers={"User-Agent": "Mozilla/5.0"}).text
soup2 = BeautifulSoup(html, "lxml")

blocks = soup2.find_all("div", class_="courseblock")
rows = []
for b in blocks:
    t = b.find("p", class_="courseblocktitle")
    if not t:
        continue
    raw = t.get_text(" ", strip=True)
    if not (raw.startswith("DATA") or raw.startswith("STAT")):
        continue
    s = t.find("span")
    if s:
        s.extract()
    course = t.get_text(" ", strip=True)
    term = ""
    for p in b.find_all("p", class_="noindent"):
        txt = p.get_text(" ", strip=True)
        if txt.lower().startswith("term typically offered"):
            term = txt.split(":", 1)[-1].strip()
            break
    rows.append({"course": course, "term_typically_offered": term})

df_courses = pd.DataFrame(rows, columns=["course","term_typically_offered"])
df_courses.shape, df_courses.head(12)


((74, 2),
                                                course term_typically_offered
 0                   DATA 100. Data Science for All I.               F, W, SP
 1             DATA 301. Introduction to Data Science.               F, W, SP
 2          DATA 401. Data Science Process and Ethics.                      F
 3   DATA 402. Mathematical Foundations of Data Sci...                      F
 4         DATA 403. Data Science Projects Laboratory.                      F
 5                DATA 441. Bioinformatics Capstone I.                      W
 6               DATA 442. Bioinformatics Capstone II.                     SP
 7                  DATA 451. Data Science Capstone I.                      W
 8                 DATA 452. Data Science Capstone II.                     SP
 9                     DATA 472. Data Science Seminar.               F, W, SP
 10   STAT 130. Introduction to Statistical Reasoning.               F, W, SP
 11  STAT 150. Introduction to the Discipline of St...

Hints:

- Each course is represented by a `<div>` with `class=courseblock`, so you can find all the courses with `soup.find_all("div", {"class": "courseblock"})`
- The course name is in a `<p>` tag with `class=courseblocktitle`, inside a `<strong>` tag. (Though I don't think we need to find the strong tag here.)
- The term typically offered is in `<p>` tag with `class=noindent`. However, there are several tags with this class; term typically offered is the first one.
- If you want to use Beautiful Soup to remove the course units (e.g., 4 units), find the `<span>` tag within the course name tag and `.extract()` this span tag